In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import h5py
import glob

### Time series of run

In [7]:
data_path = 'Runs/Example_run/time.h5'

In [8]:
data_file = h5py.File(data_path, 'r')
ts = data_file['series']
iterations = ts[:,0]
time = ts[:,1]

### Snapshots 

In [20]:
data_path = 'Runs/Example_run'
snapshot_list = glob.glob(data_path + '/*.npz')
snapshot_list.sort()

In [23]:
# load certain/s snapshot from the list
snapshot_data = np.load(snapshot_list[-1])
temperature = snapshot_data['T']
vx = snapshot_data['vx']
vz = snapshot_data['vz']
v = (vx**2+vz**2)**0.5
p = snapshot_data['p']